<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%203/Modulo_3_Ejercicio_4_Resuelto_txt_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 4: Clustering No Supervisado en el Sector Automotriz

## Objetivo
En este ejercicio, utilizarás el dataset **control_calidad_automotriz.csv** para aplicar técnicas de clustering no supervisado (k-means) y segmentar los registros en grupos basados en sus características numéricas. Aprenderás a:

- Cargar y explorar el dataset.
- Seleccionar y escalar las variables numéricas relevantes.
- Determinar el número óptimo de clusters mediante el método del codo.
- Ejecutar k-means para segmentar los datos.
- Visualizar los clusters usando una reducción de dimensionalidad (PCA).

¡Comencemos!

## 1. Importación de Librerías y Carga del Dataset

Importa las librerías necesarias y carga el dataset **control_calidad_automotriz.csv**. Se asume que el archivo se encuentra en el mismo directorio que este notebook.

Para este ejercicio, se asumirán que el dataset contiene las siguientes columnas relevantes:
- **ID**: Identificador del registro
- **Calidad**: Puntuación numérica de calidad
- **Defectos**: Cantidad de defectos detectados
- **Durabilidad**: Medida numérica que indica la durabilidad del producto
- **Eficiencia**: Indicador numérico de eficiencia en el proceso


In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset de calidad en el sector automotriz
df_auto = pd.read_csv('control_calidad_automotriz.csv')

# Mostrar las primeras filas para verificar la carga
print("Primeras filas del dataset automotriz:")
display(df_auto.head())

# Mostrar información general del DataFrame
print("Información del dataset automotriz:")
display(df_auto.info())

# Mostrar estadísticas descriptivas
print("Estadísticas descriptivas del dataset automotriz:")
display(df_auto.describe())

## 2. Selección y Escalado de Variables Numéricas

Selecciona las variables numéricas relevantes para el clustering. En este caso, utilizaremos las columnas **Calidad**, **Defectos**, **Durabilidad** y **Eficiencia**. Luego, aplicaremos escalado a estas variables para que todas tengan la misma importancia en el análisis.

Utilizaremos `StandardScaler` para normalizarlas.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Seleccionar variables numéricas
numeric_features = ['Calidad', 'Defectos', 'Durabilidad', 'Eficiencia']
df_numeric = df_auto[numeric_features].copy()

# Aplicar StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=numeric_features)

print("Variables numéricas escaladas:")
display(df_scaled.head())

## 3. Determinación del Número Óptimo de Clusters (Método del Codo)

Para determinar el número óptimo de clusters, utilizaremos el método del codo. Este método implica ejecutar k-means con distintos valores de k y graficar la inercia (suma de las distancias cuadráticas dentro del cluster). El "codo" de la gráfica indica un buen compromiso entre complejidad y ajuste.

In [ ]:
from sklearn.cluster import KMeans

# Probar diferentes valores de k y almacenar la inercia
inercia = []
k_values = range(1, 11)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inercia.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 4))
plt.plot(k_values, inercia, marker='o')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para determinar el número óptimo de clusters')
plt.show()

## 4. Aplicación de k-means y Visualización de Clusters

Una vez determinado un valor óptimo de k (por ejemplo, supongamos k=3), aplica el algoritmo k-means a los datos escalados. Para visualizar los clusters, reduciremos la dimensionalidad a 2 componentes usando PCA y representaremos los grupos en un scatter plot.

In [ ]:
# Definir el número óptimo de clusters (ajusta según el método del codo)
k_optimo = 3
kmeans = KMeans(n_clusters=k_optimo, random_state=42)
clusters = kmeans.fit_predict(df_scaled)

# Agregar la etiqueta de cluster al DataFrame escalado
df_scaled['Cluster'] = clusters

# Aplicar PCA para reducir a 2 componentes
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
df_pca = pd.DataFrame(pca.fit_transform(df_scaled[numeric_features]), columns=['PC1', 'PC2'])
df_pca['Cluster'] = clusters

# Visualizar los clusters en el espacio de los dos primeros componentes
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Cluster', data=df_pca, palette='viridis', s=100, alpha=0.8)
plt.title('Visualización de Clusters mediante PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.show()

# Mostrar la cantidad de registros en cada cluster
print(df_scaled['Cluster'].value_counts())

## 5. Sistema de Testeo Automático

Este bloque de código verificará que:

- El DataFrame original no esté vacío.
- Las variables numéricas seleccionadas existen en el DataFrame.
- El método del codo ha generado una lista de inercia con la misma longitud que el rango de k probado.
- El DataFrame resultante del PCA tiene 2 componentes.

In [ ]:
# Test 1: Verificar que el DataFrame original no está vacío
assert df_auto.shape[0] > 0, "Test fallido: El DataFrame original está vacío."

# Test 2: Verificar que las columnas numéricas existen
for col in ['Calidad', 'Defectos', 'Durabilidad', 'Eficiencia']:
    assert col in df_auto.columns, f"Test fallido: La columna {col} no se encontró en el DataFrame."

# Test 3: Verificar que se generó la lista de inercia para k de 1 a 10
assert len(inercia) == 10, "Test fallido: La lista de inercia no tiene la longitud esperada (10)."

# Test 4: Verificar que el DataFrame del PCA tiene 2 componentes
assert df_pca.shape[1] == 3, "Test fallido: El DataFrame del PCA debería tener 3 columnas (PC1, PC2 y Cluster)."

print("Todos los tests se han pasado correctamente.")

## 6. Reflexión Final

1. ¿Qué insights te aporta la segmentación de los datos mediante clustering?
2. ¿Cómo interpretas los clusters obtenidos en el contexto del control de calidad en el sector automotriz?
3. ¿Qué mejoras o ajustes propondrías para optimizar la segmentación?

_Responde estas preguntas en una celda Markdown adicional o en un comentario._